In [5]:
from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh,
                           xywh2xyxy)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

from KalmanFilter import KalmanFilter
#from SumTensorList import sum_tensor

In [6]:
KF = KalmanFilter((1 /10), 1, 1, 1, 0.1,0.1)

In [7]:
(X,Y) = KF.predict()
Last_whcc = torch.zeros((1, 4), device='cuda').view(-1)
X = (torch.tensor(X, device="cuda")).view(-1)
Y = (torch.tensor(Y, device="cuda")).view(-1)
print(X)
print(Last_whcc)
print(X.shape)
print(Last_whcc.shape)

tensor([0.00500], device='cuda:0', dtype=torch.float64)
tensor([0., 0., 0., 0.], device='cuda:0')
torch.Size([1])
torch.Size([4])


In [20]:
pred = [torch.tensor([[2.49784e+02, 3.63123e+01, 4.66263e+02, 3.21688e+02, 2.71220e-01, 0.00000e+00]], device='cuda')]

pred2 = [torch.tensor([[2.49784e+02, 3.63123e+01, 4.66263e+02, 3.21688e+02, 2.71220e-01, 0.00000e+00],[2.49784e+02, 3.63123e+01, 4.66263e+02, 3.21688e+02, 2.71220e-01, 0.00000e+00]],
                      device='cuda')]
pred3 = [torch.zeros((0, 6), device='cuda')] 
Zero_pred = torch.zeros((0, 6), device='cuda')
zero_whcc = torch.zeros((1, 4), device='cuda').view(-1)
Last_whcc = 0.5 * torch.ones((1, 4), device='cuda').view(-1)
Last_xy=[[200], [150]]
print(pred2)

im0s= 1
#[tensor([[4.20552e+02, 1.29332e+02, 5.52588e+02, 2.39458e+02, 3.78457e-01, 0.00000e+00]], device='cuda:0')]
#[tensor([3.58023e+02, 1.79000e+02, 2.16479e+02, 2.85376e+02, 2.71220e-01, 0.00000e+00], device='cuda:0'), tensor([3.58023e+02, 1.79000e+02, 2.16479e+02, 2.85376e+02, 2.71220e-01, 0.00000e+00], device='cuda:0')]


[tensor([[2.49784e+02, 3.63123e+01, 4.66263e+02, 3.21688e+02, 2.71220e-01, 0.00000e+00],
        [2.49784e+02, 3.63123e+01, 4.66263e+02, 3.21688e+02, 2.71220e-01, 0.00000e+00]], device='cuda:0')]


In [12]:
#Yolov5로 예측을 성공한 경우

li=[]


for i, d in enumerate(pred2):
    #yolo모델이 검출에 실패한 경우 
    #칼만필터로 예측 진행
    
    if torch.equal(d, Zero_pred) and not(torch.equal(zero_whcc, Last_whcc)) :
 
        #칼만필터로 예측된 값을 텐서 변환
        X = (torch.tensor(X, device="cuda")).view(-1)
        Y = (torch.tensor(Y, device="cuda")).view(-1)

        #예측 좌표와 이전 프레임의 값을 가져와 (1,6)의 텐서로 결합
        d = torch.cat([X, Y, Last_whcc], dim=0)
                
        #텐서의 차원 맞추기
        d = d.view(1,-1)
        
    #칼만 혹은 yolo에서 예측된 텐서를 보정
    for *xyxy, conf, clas in reversed(d):
        # xywh 형태로 변형
        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4))).view(-1).tolist() #List 형태로 반환

        
        #텐서의 shape 맞추기
        conf = conf.view(-1)
        clas = clas.view(-1)

        # 지난 프레임의 예측 값을 측정값으로 사용하여 
        # 칼만필터의 추정값을 계산
        est_x_y = KF.update(Last_xy)        
        
        # 칼만필터에 input 형태로 변형
        Last_xy = [[xywh[0]], [xywh[1]]]
        
        # 칼만필터의 추정값을 텐서에 적용
        xywh[0] = est_x_y[0, 0]
        xywh[1] = est_x_y[1, 0]

        #높이와 너비값 저장
        wh = (torch.tensor(xywh[2:4], device="cuda"))
        Last_whcc = torch.cat([wh, conf, clas], dim=0)

        # Yolo의 출력 형태로 텐서 길이로 변경
        xywh = [xywh]
        xyxy = (xyxy2xywh(torch.tensor(xyxy, device="cuda").view(1, 4))).view(-1)
            
        # 원래 형태로 만들어주기
        Esti_X = torch.cat([xyxy, conf, clas], dim=0)
        
        # 텐서를 차원을 맞추어 리스트에 저장
        
        Esti_X = Esti_X.unsqueeze(0)
        li.append(Esti_X)
        
        
#if li is not None:
    
    #pred4 =[sum_tensor(li)]

print(li)



[tensor([[3.58023e+02, 1.79000e+02, 2.16479e+02, 2.85376e+02, 2.71220e-01, 0.00000e+00]], device='cuda:0'), tensor([[3.58023e+02, 1.79000e+02, 2.16479e+02, 2.85376e+02, 2.71220e-01, 0.00000e+00]], device='cuda:0')]


In [17]:
Zero_pred = torch.zeros((0, 6), device='cuda')
zero_whcc = torch.zeros((1, 4), device='cuda').view(-1)
Last_whcc = 0.5 * torch.ones((1, 4), device='cuda').view(-1)
Last_x_y = [[0], [0]]


In [16]:
#Yolov5로 예측을 성공한 경우

li=[]


for i, d in enumerate(pred2):
    #yolo모델이 검출에 실패한 경우 
    #칼만필터로 예측 진행
    
    print(d)
    #칼만 혹은 yolo에서 예측된 텐서를 보정
    for *xyxy, conf, clas in reversed(d):
        
        
        # xywh 형태로 변형
        
        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4))).view(-1).tolist() #List 형태로 반환

        
        #텐서의 shape 맞추기
        conf = conf.view(-1)
        clas = clas.view(-1)

        # 지난 프레임의 예측 값을 측정값으로 사용하여 
        # 칼만필터의 추정값을 계산
        est_x_y = KF.update(Last_xy)        
        
        # 칼만필터에 input 형태로 변형
        Last_xy = [[xywh[0]], [xywh[1]]]
        
        # 칼만필터의 추정값을 텐서에 적용
        xywh[0] = est_x_y[0, 0]
        xywh[1] = est_x_y[1, 0]

        #높이와 너비값 저장
        wh = (torch.tensor(xywh[2:4], device="cuda"))
        Last_whcc = torch.cat([wh, conf, clas], dim=0)

        # Yolo의 출력 형태로 텐서 길이로 변경
        xywh = [xywh]
        xyxy = (xyxy2xywh(torch.tensor(xyxy, device="cuda").view(1, 4))).view(-1)
            
        # 원래 형태로 만들어주기
        Esti_X = torch.cat([xyxy, conf, clas], dim=0)
        
        # 텐서를 차원을 맞추어 리스트에 저장
        
        Esti_X = Esti_X.unsqueeze(0)
        li.append(Esti_X)
if li is not None:
    
    pred4 =[sum_tensor(li)]
    
    print(pred4)



tensor([[2.49784e+02, 3.63123e+01, 4.66263e+02, 3.21688e+02, 2.71220e-01, 0.00000e+00],
        [2.49784e+02, 3.63123e+01, 4.66263e+02, 3.21688e+02, 2.71220e-01, 0.00000e+00]], device='cuda:0')


NameError: name 'sum_tensor' is not defined

In [39]:
li = []

# Process predictions
for i, det in enumerate(pred2):  # per image
    
    
    if not len(det):
        # 칼만필터로 예측된 값을 텐서 변환
        (X,Y) = KF.predict()
        X = (torch.tensor(X, device="cuda")).view(-1)
        Y = (torch.tensor(Y, device="cuda")).view(-1)
          
        # 예측 좌표와 이전 프레임의 값을 가져와 (1,6)의 텐서로 결합
        xywh = torch.cat([X, Y, Last_whcc[0:2]], dim=0).tolist()
        xyxy = (xywh2xyxy(torch.tensor(xywh, device="cuda").view(1, 4))).view(-1)
        det = torch.cat([xyxy, Last_whcc[2:4]], dim=0).unsqueeze(0)
        
        print(det)
        print(det.shape)
    
    
    
    for *xyxy, conf, clas in reversed(det):
        print(xyxy)
        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4))).view(-1).tolist()
        print(xywh)
        conf = conf.view(-1)
        clas = clas.view(-1)  
        # 지난 프레임의 예측 값을 측정값으로 사용하여
        # 칼만필터의 추정값을 계산

        est_x_y = KF.update(Last_x_y)
        print(est_x_y)
    
        # 칼만필터에 input 형태로 변형
        Last_x_y = [[xywh[0]], [xywh[1]]]
    
        # 칼만필터의 추정값을 텐서에 적용
        xywh[0] = est_x_y[0, 0]
        xywh[1] = est_x_y[1, 0]

        # 높이와 너비값 저장
    
        wh = (torch.tensor(xywh[2:4], device="cuda"))
        Last_whcc = torch.cat([wh, conf, clas], dim=0)

        # Yolo의 출력 형태로 텐서 길이로 변경
        xywh = [xywh]
        xyxy = (xywh2xyxy(torch.tensor(xywh, device="cuda").view(1, 4))).view(-1)
             
        # 원래 형태로 만들어주기
        Esti_X = torch.cat([xyxy, conf, clas], dim=0)

        # 텐서를 차원을 맞추어 리스트에 저장

        Esti_X = Esti_X.unsqueeze(0)
    
        li.append(Esti_X)
if li is not None:
    det = sum_tensor(li)
print(det)

[tensor(249.78400, device='cuda:0'), tensor(36.31230, device='cuda:0'), tensor(466.26300, device='cuda:0'), tensor(321.68799, device='cuda:0')]
[358.02349853515625, 179.00015258789062, 216.47900390625, 285.37567138671875]
[[        312]
 [        155]]
[tensor(249.78400, device='cuda:0'), tensor(36.31230, device='cuda:0'), tensor(466.26300, device='cuda:0'), tensor(321.68799, device='cuda:0')]
[358.02349853515625, 179.00015258789062, 216.47900390625, 285.37567138671875]
[[        318]
 [        158]]
tensor([[2.03760e+02, 1.23122e+01, 4.20240e+02, 2.97688e+02, 2.71220e-01, 0.00000e+00],
        [2.09760e+02, 1.53122e+01, 4.26240e+02, 3.00688e+02, 2.71220e-01, 0.00000e+00]], device='cuda:0', dtype=torch.float64)


In [22]:
print(li)

[tensor([[ 1.43760e+02, -1.86878e+01,  3.60240e+02,  2.66688e+02,  2.71220e-01,  0.00000e+00]], device='cuda:0', dtype=torch.float64), tensor([[ 1.69760e+02, -5.68784e+00,  3.86240e+02,  2.79688e+02,  2.71220e-01,  0.00000e+00]], device='cuda:0', dtype=torch.float64)]


In [23]:
for i in li:
    print(i)

tensor([[ 1.43760e+02, -1.86878e+01,  3.60240e+02,  2.66688e+02,  2.71220e-01,  0.00000e+00]], device='cuda:0', dtype=torch.float64)
tensor([[ 1.69760e+02, -5.68784e+00,  3.86240e+02,  2.79688e+02,  2.71220e-01,  0.00000e+00]], device='cuda:0', dtype=torch.float64)


In [34]:
def sum_tensor(li):
    a = torch.zeros((0, 6), device='cuda')
    for i in li:
        a = torch.cat([a , i] , dim=0)
        
    return a

In [35]:
print(li)

[tensor([[ 1.43760e+02, -1.86878e+01,  3.60240e+02,  2.66688e+02,  2.71220e-01,  0.00000e+00]], device='cuda:0', dtype=torch.float64), tensor([[ 1.69760e+02, -5.68784e+00,  3.86240e+02,  2.79688e+02,  2.71220e-01,  0.00000e+00]], device='cuda:0', dtype=torch.float64)]


In [36]:
det = sum_tensor(li)

In [37]:
print(det)

tensor([[ 1.43760e+02, -1.86878e+01,  3.60240e+02,  2.66688e+02,  2.71220e-01,  0.00000e+00],
        [ 1.69760e+02, -5.68784e+00,  3.86240e+02,  2.79688e+02,  2.71220e-01,  0.00000e+00]], device='cuda:0', dtype=torch.float64)
